In [13]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, min, year,count,when,sum,round

spark = SparkSession \
    .builder \
    .config("spark.jars",
            "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
    .master("local[*]") \
    .appName("leetcode-problems") \
    .getOrCreate()

orders = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/namastesql") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "orders") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

customers = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/namastesql") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "customers") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [15]:
customers\
    .join(orders,customers.order_id == orders.order_id,how="right")\
    .select(orders.customer_name,orders.order_id,customers.return_date)\
    .withColumn("return_check",when(col("return_date").isNotNull(),1))\
    .groupby("customer_name")\
    .agg(count("order_id").alias("total_orders"),
         sum(when(col("return_check")==1,col("return_check")).otherwise(0)).alias("return_count"))\
    .withColumn("return_rate",round((col("return_count")/col("total_orders"))*100,2))\
    .where(col("return_rate")>50)\
    .select("customer_name","return_rate")\
    .show()

+-------------+-----------+
|customer_name|return_rate|
+-------------+-----------+
|        Priya|      66.67|
|        Ankit|      100.0|
|        Alexa|       75.0|
+-------------+-----------+

